最近开始想自己做点实际的深度学习项目了，奈何没有GPU啊。找老板配了一台1080Ti的主机，送来预装的是Win10系统。虽然在Win下也可以搞深度学习，但总觉得没有跟上潮流，而且各种软件的安装管理确实也没有Ubuntu下方便，于是装双系统，在Ubuntu环境下搭建CUDA，cuDNN，TensorFlow深度学习环境。

以下是具体步骤记录，防止以后重复造轮子

20190515 Update:

备选方案
[Ubuntu18.04深度学习环境配置(简易方式)](https://mp.weixin.qq.com/s/RNnAtnkfejH_fS1wPGG4Hw)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Ubuntu环境下CUDA-cuDNN-TensorFlow安装" data-toc-modified-id="Ubuntu环境下CUDA-cuDNN-TensorFlow安装-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Ubuntu环境下CUDA-cuDNN-TensorFlow安装</a></span><ul class="toc-item"><li><span><a href="#第一步：安装nvdia-driver" data-toc-modified-id="第一步：安装nvdia-driver-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>第一步：安装nvdia driver</a></span></li><li><span><a href="#第二步：安装CUDA" data-toc-modified-id="第二步：安装CUDA-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>第二步：安装CUDA</a></span><ul class="toc-item"><li><span><a href="#下载-CUDA" data-toc-modified-id="下载-CUDA-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>下载 CUDA</a></span></li><li><span><a href="#运行.run文件进行安装" data-toc-modified-id="运行.run文件进行安装-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>运行<code>.run</code>文件进行安装</a></span></li><li><span><a href="#设置环境变量" data-toc-modified-id="设置环境变量-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>设置环境变量</a></span></li><li><span><a href="#检查-CUDA-是否安装成功" data-toc-modified-id="检查-CUDA-是否安装成功-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>检查 CUDA 是否安装成功</a></span></li></ul></li><li><span><a href="#安装-cnDNN" data-toc-modified-id="安装-cnDNN-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>安装 cnDNN</a></span><ul class="toc-item"><li><span><a href="#下载-cnDNN" data-toc-modified-id="下载-cnDNN-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>下载 cnDNN</a></span></li></ul></li><li><span><a href="#安装-TensorFlow" data-toc-modified-id="安装-TensorFlow-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>安装 TensorFlow</a></span></li><li><span><a href="#已知问题" data-toc-modified-id="已知问题-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>已知问题</a></span></li></ul></li></ul></div>

# Ubuntu环境下CUDA-cuDNN-TensorFlow安装

由于目前TensorFlow只支持CUDA9.0，故安装CUDA9.0+TensorFlow组合，如果安装了太新的CUDA会导致TensorFlow无法import

网上有一些提前编译的适用CUDA9.2的TF whl，但是为了稳定，不尝试

## 第一步：安装nvdia driver

最简单的方法就是在 Ubuntu的“软件与更新”里安装驱动如图

<div align="center">  <img src="http://image-bed-zodiac.oss-cn-qingdao.aliyuncs.com/18-9-12/89437707.jpg" width="500"/> </div><br>

## 第二步：安装CUDA

### 下载 CUDA

从[CUDA Archive](https://developer.nvidia.com/cuda-toolkit-archive) 下载对应版本的CUDA安装包，我使用的是 run file

<div align="center">  <img src="http://image-bed-zodiac.oss-cn-qingdao.aliyuncs.com/18-9-12/52507130.jpg" width="500"/> </div><br>

### 运行`.run`文件进行安装

```shell
sudo sh cuda_9.0.176_384.81_linux.run
```

过程中会有一些选项

```shell
You are attempting to install on an unsupported configuration. Do you wish to continue?
(y)es/(n)o [ default is no ]: y

Install NVIDIA Accelerated Graphics Driver for Linux-x86_64 396.26?
(y)es/(n)o/(q)uit: n

Install the CUDA 9.0 Toolkit?
(y)es/(n)o/(q)uit: y

Enter Toolkit Location
 [ default is /usr/local/cuda-9.0 ]:

Do you want to install a symbolic link at /usr/local/cuda?
(y)es/(n)o/(q)uit: y

Install the CUDA 9.0 Samples?
(y)es/(n)o/(q)uit: y

Enter CUDA Samples Location
 [ default is /home/kinghorn ]: /usr/local/cuda-9.0
```

**一定要注意第二个选项 是否安装 NVIDIA Accelerated Graphics Driver，选择 n 不要安装**因为我们已经使用Ubuntu驱动管理安装过显卡驱动了

之后再将剩下的补丁也装上

```shell
sudo sh cuda_9.0.176.1_linux.run
```

### 设置环境变量

```shell
$ echo 'export PATH=/usr/local/cuda-9.0/bin${PATH:+:${PATH}}' >> ~/.bashrc

# 64位系统

$ echo 'export LD_LIBRARY_PATH=/usr/local/cuda-9.0/lib64${LD_LIBRARY_PATH:+:${LD_LIBRARY_PATH}}' >> ~/.bashrc

$ source ~/.bashrc # 更新PATH
```

重启电脑

输入命令

```shell
$ nvidia-smi
```
会显示显卡信息

### 检查 CUDA 是否安装成功

1 检查 cuda-toolkit 是否安装成功

```shell
nvcc -V
```

应该会输出 cuda 版本信息

2 运行 sample

进入 sample 文件夹

```shell
make
```

**已知问题**：nvcc依赖gcc<6 但是目前系统的gcc版本太高，需要安装低版本gcc并链接

```shell
$ sudo ln -s /usr/bin/gcc-4.9 /usr/local/cuda/bin/gcc

$ sudo ln -s /usr/bin/g++-4.9 /usr/local/cuda/bin/g++
```

一切正常的话，应该编译通过

运行deviceQuery 和 bandwidthTest 程序

## 安装 cnDNN

### 下载 cnDNN

从[cnDNN Archive](https://developer.nvidia.com/rdp/cudnn-archive)下载对应版本cnDNN

解压文件，将指定文件移动到 CUDA 文件夹并给予权限

```shell
$ tar -xzvf cudnn-9.0-linux-x64-v7.tgz

$ sudo cp cuda/include/cudnn.h /usr/local/cuda/include

$ sudo cp cuda/lib64/libcudnn* /usr/local/cuda/lib64

$ sudo chmod a+r /usr/local/cuda/include/cudnn.h /usr/local/cuda/lib64/libcudnn*
```

## 安装 TensorFlow

使用 conda env 安装

切换到相应环境，直接安装

```shell
source activate mypython3 # 激活Python环境，如果没有新建虚拟环境可省略此步骤

pip install tensorflow-gpu --ignore-installed --upgrade
```

由于权限问题，可能需要使用`--user`选项

```shell
pip install tensorflow-gpu --ignore-installed --upgrade --user
```


References:

>- [How to install CUDA 9.2 on Ubuntu 18.04](https://www.pugetsystems.com/labs/hpc/How-to-install-CUDA-9-2-on-Ubuntu-18-04-1184/)
- https://docs.nvidia.com/cuda/cuda-installation-guide-linux/index.html
- https://docs.nvidia.com/deeplearning/sdk/cudnn-install/index.html#install-linux

## 已知问题

安装cuda的时候，提示
toolkit installation failed using unsupported compiler 这是因为GCC版本不合适

解决的方法 直接在安装命令之后加-override

```shell
sudo sh cuda_9.0.176_384.81_linux.run -override
```

在 python 中 import tensorflow 提示

ImportError: libcublas.so.9.0: cannot open shared object file: No such file or directory

一般是因为 cuda 版本有问题，比如目前TF只支持CUDA9.0 却安装了CUDA9.2

或是cuda没有添加到PATH中
```shell

export PATH=/usr/local/cuda-9.0/bin${PATH:+:${PATH}}
export LD_LIBRARY_PATH=LD_LIBRARY_PATH:/usr/local/cuda-9.0/lib64/
```

如何完全卸载 nvidia driver [How can I uninstall a nvidia driver completely ?](https://askubuntu.com/questions/206283/how-can-i-uninstall-a-nvidia-driver-completely)